In [1]:
import pandas as pd

portfolio = pd.read_excel("data/portfolio.xlsx", sheet_name="Stocks")

Let's update Company Name, Market Price and Annual Dividend per Share
Notes:
- Market Price can be extracted from <ticker>.info["currentPrice"] for regular stocks, but not for ETFs
- For ETF there is <ticker>.info["navPrice"], which may slightly differ comparing to market price
- ETF net asset is a value of all assets of the fund, substracting liabilities. For ETFs NAV price is calculated every day
- Stocks dividends can be returned using dividendRate, but not for ETF. Ideally for ETFs we should calculate TTM dividend rate (Trailing twelwe months)
- yfinance does have <ticker>.info["trailingAnnualDividendRate"] but it's 0 for SCHD. Not reliable

In [2]:
import yfinance as yf
from main import (
    calculate_etf_ttm,
    calculate_percentage_difference,
    dividends_paid_per_share,
    calculate_total_return
)

for i in range(len(portfolio)):
  ticker_data = yf.Ticker(portfolio.loc[i, "Ticker"])
  match ticker_data.info["quoteType"]:
    case "ETF":
      annual_dividend_per_share = calculate_etf_ttm(ticker_data)
      market_price = round(ticker_data.info["navPrice"], 2)
    case _:
      annual_dividend_per_share = ticker_data.info["dividendRate"]
      market_price = round(ticker_data.info["currentPrice"],2)

  buy_price = portfolio.loc[i, "Buy Price"]
  capital_gains = calculate_percentage_difference(buy_price,market_price)
  num_of_shares = portfolio.loc[i, "Number of Shares"]
  sum_of_divs_per_share = dividends_paid_per_share(portfolio.loc[i, "Date of Purchase"],ticker_data)
  total_return = calculate_total_return(sum_of_divs_per_share,buy_price,market_price)

  portfolio.loc[i, "Company Name"] = ticker_data.info["shortName"]
  portfolio.loc[i, "Market Price"] = market_price
  portfolio.loc[i, "Annual Dividend per Share"] = annual_dividend_per_share
  portfolio.loc[i, "Capital Gains"] = round(capital_gains, 2)
  portfolio.loc[i, "Dividends Paid"] = sum_of_divs_per_share * num_of_shares
  portfolio.loc[i, "Total Return"] = total_return



/var/folders/nm/f3npn9kn4zv1vmr6y6m8gyt80000gn/T/ipykernel_44319/3459206826.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '611.4300000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  portfolio.loc[i, "Dividends Paid"] = sum_of_divs_per_share * num_of_shares


In [3]:
portfolio.head()

,Company Name,Ticker,Number of Shares,Buy Price,Market Price,Annual Dividend per Share,Date of Purchase,Capital Gains,Dividends Paid,Total Return
0,Atlantica Sustainable Infrastru,AY,687,23.55,21.60,1.78,22.06.2023,-8.28,611.43,-4.501062
1,Atlantica Sustainable Infrastru,AY,400,24.66,21.60,1.78,27.07.2023,-12.41,356.00,-8.799676
2,Schwab US Dividend Equity ETF,SCHD,983,75.65,75.82,2.66,27.07.2023,0.22,1376.20,2.075347


In [78]:
portfolio.to_excel("data/portfolio.xlsx", sheet_name="Stocks", index=False)